# Audio Day 4 - Audio Features

## Mel Frequency Cepstral Coefficients (MFCCs)

MFCCs are by far the most commonly used audio feature in machine learning. You can build good models with nothing but MFCCs.

### Mel scale

The Mel-scale is a perceptual scale of pitch judged by listeners to be equidistant.

It aims to mimic the non-linear human ear perception of sound. 

The name mel comes from the word melody to indicate that the scale is based on pitch comparisons.

The frequency-mel conversion is based on results of experiments on human subjects, so there is no objectively correct formula.

Example formula to convert from frequency to mel:

$m = 2595 \log_{10}(1+\frac{f}{700})$

Mel to frequency:

$f = 700(10^{\frac{m}{2595}}-1)$

![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Mel-Hz_plot.svg/1920px-Mel-Hz_plot.svg.png)

Let's take a look at how a linear scale spectrogram translates to a mel scale spectrogram in Audacity.

### Mel Frequency Cepstral Coefficients (MFCCs)

Like formants, MFCCs are another way of representing information about the deformation of the human vocal tract. They are especially useful for machine learning.

To calculate the MFCCs, we do the following:

1. Divide the signal into frames
2. Apply the Fourier Transform to create the spectrum of each frame
3. Apply a mel filterbank
4. Take the log of filterbank
5. Apply the discrete cosine transform

Depending on how many filters are used, we get up to 40 cepstral coefficents for each frame.

Mel filterbank

Each filter in a Mel filterbank has a triangular shape and can be a applied to a spectrum  by multiplying the amplitude by 1 at the center of the filter, all the way down to 0 at the edges of the filter.

![](https://haythamfayek.com/assets/posts/post1/mel_filters.jpg)
Source: [Haytham Fayek](https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html)

![](http://practicalcryptography.com/media/miscellaneous/files/mel_filterbank_example.png)

Source: [James Lyons](http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/)

MFCCs are widely used in speech recognition, musical genre classification, speaker clustering, emotion recognition, etc.

The first 12-20 MFCCs are often considered to carry enough discriminating information in the context of various classification tasks.

### Extracting MFCCs

In addition to representing audio signals in Python, our primary use of Librosa is [feature extraction](https://librosa.org/doc/latest/feature.html).

In [ ]:
# let's work with the [i] again
!wget https://upload.wikimedia.org/wikipedia/commons/9/91/Close_front_unrounded_vowel.ogg
!ffmpeg -i "Close_front_unrounded_vowel.ogg" "i.wav" -y

In [ ]:
# load the data
import librosa
y, sr = librosa.load("i.wav", mono = True)

In [ ]:
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
print(mfcc.shape)
print(mfcc)

In [ ]:
import librosa.display
librosa.display.specshow(mfcc)

Let's investigate what's going on under the hood and look at the Librosa documentation and source code for [mfcc](https://librosa.org/doc/latest/generated/librosa.feature.mfcc.html) and [melspectrogram](https://librosa.org/doc/latest/generated/librosa.feature.melspectrogram.html).

In [ ]:
 # Let's start with a regular spectrogram
import numpy as np
import librosa.display
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
librosa.display.specshow(D, y_axis = 'linear')

In [ ]:
# Produce a linear transformation matrix 
# to project FFT bins onto Mel-frequency bins
n_fft = 2048
mel_basis = librosa.filters.mel(sr=sr, n_fft=n_fft)

In [ ]:
# Illustrate the transformation matrix
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
img = librosa.display.specshow(mel_basis, x_axis='linear', ax=ax)
ax.set(ylabel='Mel filter', title='Mel filter bank')
fig.colorbar(img, ax=ax)

In [ ]:
# Transform the spectrogram to mel scale
D_mel = np.einsum("...ft,mf->...mt", D, mel_basis, optimize=True)
# and take a look at it
librosa.display.specshow(D_mel, x_axis='linear')

In [ ]:
D_mel.shape

In [ ]:
# Apply the DCT
import scipy
n_mfcc=20
M = scipy.fftpack.dct(D_mel, axis=-2, norm="ortho")[..., :n_mfcc, :]

In [ ]:
# finally, we get our MFCCs
M.shape

In [ ]:
librosa.display.specshow(M)

Here's another well-documented extraction [function](https://github.com/jameslyons/python_speech_features/blob/master/python_speech_features/base.py).

### Delta MFCC features

Sound is a moving signal. Features like MFCCs are only taken at static points in time. **Delta features** are used to measure the change in features between two frames. So for example, the delta MFCCs of two frames are simply the MFCCs of the second frame minus the MFCCs of the first frame.

In [ ]:
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
delta = librosa.feature.delta(mfcc)

In [ ]:
print(mfcc)
print(delta)
print(mfcc.shape)
print(delta.shape)

## Other features

### Pitch

Pitch is the subjective human perception of the fundamental frequency (F0).

The higher the fundamental frequency, the higher the pitch.

The relationship between the fundamental frequency and pitch is not linear because our hearing does not represent frequency differences below 100Hz and above 1000Hz accurately.

For example, to the human ear, the difference between 900Hz and 1000Hz sounds larger than the difference between 1000Hz and 1100Hz.

Between 100Hz and 1000Hz, the fundamental frequency and pitch correlate linearly. Above, they correlatey logarithmically.

Since pitch is subjective, it can't be measured objectively in the way we measure physics-based sound properties.

#### Pitch Extraction

The features we discussed above relate to the amplitude and occur at the sample level. They are time-domain audio features.

Pitch is measured at the FFT (Fast Fourier Transform) frame level. It is a frequency-domain audio feature.

These frames are applied to the audio track at very small, overlapping intervals.

##### Librosa

Librosa has an algorithm for tracking pitch. Unfortunately it doesn't work very well.

In [ ]:
#Don't use this
pitch_values, magnitudes = librosa.piptrack(y=y, sr=sr)#, fmin=75, fmax=600)

In [ ]:
pitch_values

In [ ]:
import numpy as np
with np.printoptions(threshold=np.inf):
    print(pitch_values)

##### Parselmouth/Praat

Praat has its own [function](https://www.fon.hum.uva.nl/praat/manual/Sound__To_Pitch___.html) for pitch tracking, and at least for this audio track, it works better than librosa.

In [ ]:
!pip install praat-parselmouth

In [ ]:
import parselmouth
snd = parselmouth.Sound("i.wav")

In [ ]:
#Use this
pitch = snd.to_pitch()
pitch_values = pitch.selected_array['frequency']
pitch_values

If you are serious about pitch detection, you might also want to consider [Matlab](https://www.mathworks.com/help/audio/ref/pitch.html). Wesleyan students get to use it for free and you can also use an online version at https://myapps.wesleyan.edu.

##### Exercise

1. Measure the pitch
2. Use Audacity to increase the pitch by 30%
3. Measure the pitch again and compare

Use the following audio clip:

In [ ]:
!gdown https://drive.google.com/uc?id=1-9zXUFVRdUMPltiN8pPcydBd8SJr93Sy

In [ ]:
snd = parselmouth.Sound("harris_speech.wav")
pitch = snd.to_pitch()
pitch_values = pitch.selected_array['frequency']
pitch_values

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

snd = parselmouth.Sound("/content/drive/Shareddrives/qac239_prep/Audio_Section_2022_Fall/Audio_Day4/harris_speech_pitch.wav")
pitch = snd.to_pitch()
pitch_values2 = pitch.selected_array['frequency']

In [ ]:
ratios = pitch_values2/pitch_values
ratios = ratios[np.isfinite(ratios)]
ratios = ratios[ratios != 0]
ratios

In [ ]:
print(np.mean(ratios))

Examples for the use of pitch in social science research:
- Politicians who speak with lower pitch are more popular with voters ([Klofstad, Anderson & Peters 2012](https://royalsocietypublishing.org/doi/10.1098/rspb.2012.0311), [Klofstad 2016](https://web.as.miami.edu/personal/cklofstad/25_polpsych_pitch.pdf)).
- Using pitch to predict the votes of Supreme Court justices ([Dietrich, Enos & Sen 2018](https://www.cambridge.org/core/journals/political-analysis/article/emotional-arousal-predicts-voting-on-the-us-supreme-court/1047BF7D73A1B45BDB1C61A3A80E0F64)).
- Using pitch to detect which topics a political speaker cares about and wants to emphasize the most ([Dietrich, Hayes, O'Brien 2019](https://www.cambridge.org/core/journals/american-political-science-review/article/pitch-perfect-vocal-pitch-and-the-emotional-intensity-of-congressional-speech/868A227A2C57A7053742CF9CF3B6C419)).

### Measures of loudness

#### Root-mean-square amplitude

To measure the amplitude (as a measure of loudness) over a given interval, root-mean-square amplitude can be used. Rather than just taking the average amplitude, which would result in a value close to 0 because the positive and negative amplitude values would cancel each other out, we square them and then divide them by the number of samples. Then we take the root.

$N$ = number of samples \\
$i$ = sample i \\
$x_i$ = amplitude at sample i

RMS amplitude $= \sqrt{\frac{1}{N} \sum_{i=1}^{N}x^2_i}$

In [ ]:
#Librosa allows us to measure RMS amplitude like this:
librosa.feature.rms(y=y)

Librosa calculates RMS amplitude for each frame individually. In the [source code](https://librosa.org/doc/latest/_modules/librosa/feature/spectral.html#rms), the signal is divided into frames like this: \\
` x = util.frame(y, frame_length=frame_length, hop_length=hop_length)`

Beyond that, it is effectively the same formula we used (except that there is a mean for each frame): \\
`np.sqrt(np.mean(np.abs(x) ** 2, axis=-2, keepdims=True))`

Exercise:

Using the formula for RMS amplitude, implement the math yourself for `y`, and compare it to the output of the librosa function.

RMS amplitude $= \sqrt{\frac{1}{N} \sum_{i=1}^{N}x^2_i}$

In [ ]:
import numpy as np

np.sqrt(sum(y**2)/len(y))

#### Zero-crossing rate
Zero-crossing rate measures the number of times in a given time interval/frame that the amplitude of the speech signals passes through zero.

silence vs. any signal; i.e. voice activity detection

EEG

![](https://ars.els-cdn.com/content/image/3-s2.0-B9780080993881000042-f04-04-9780080993881.jpg)

#### Chroma features
12-element vector representation of the spectral energy for each pitch class in music: C, C#, D, D#, E, F, F#, G, G#, A, A# and B. Mainly used for music classification. The standard deviation is called the Chroma deviation.

Application: speech-music discrimination

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/b/b4/12tet_diatonic_scale.ogg

In [ ]:
import librosa
import librosa.display
y, sr = librosa.load("12tet_diatonic_scale.ogg")
out = librosa.feature.chroma_stft(y, sr)

fig, ax = plt.subplots()
img = librosa.display.specshow(out, y_axis='chroma', x_axis='time', ax=ax)
fig.colorbar(img, ax=ax)

#### Spectral entropy
Shannon entropy of the power spectral density. Used to measure signal irregularity.

#### Spectral flux
Squared difference between the normalized magnitudes of the spectra of the two successive short-term windows. Spectral flux tends to be higher for speech than for music. Useful for determining timbre to classify music instruments.

### Exercise
1. Use librosa to calculate the zero-crossing rate of the diatonic scale audio.
2. Calculate its delta features.
3. Plot both over time and compare.

## Homework

Librosa contains a couple of example audio tracks, among them a song called 'Fishin'. You can import it like this:



```
y, sr = librosa.load(librosa.example('fishin'))
```



1. Restrict the audio to the first 30 seconds of the song.
2. Calculate the [spectral flux](https://librosa.org/doc/latest/generated/librosa.onset.onset_strength.html?highlight=spectral%20flux#).
3. Calculate the running average spectral flux for windows of size 100. You can use the approach described [here](https://stackoverflow.com/a/54628145).
4. Make a scatterplot with time, in seconds, on the x-axis, and running average spectral flux on the y-axis.
5. Listen to the first 30 seconds of the song (you can export it to an audio file with `soundfile.write`). What do the peaks and valleys in the scatterplot correspond to?

Turn in your notebook in the Moodle dropbox by Friday, November 25, 11:59pm.

In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import librosa.display
import soundfile

In [ ]:

y, sr = librosa.load(librosa.example('fishin'), duration = 30.0)
sec30 = librosa.time_to_samples(np.array([0, 30]),sr)
y = y[sec30[0]:sec30[1]]
D = np.abs(librosa.stft(y))
times = librosa.times_like(D)
fig,ax = plt.subplots(nrows=2,sharex=True)
librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max),y_axis='log', x_axis ='time', ax=ax[0])

ax[0].set(title='Power spectrogram' )
ax[0].label_outer()
onset_env = librosa.onset.onset_strength(y=y, sr=sr)
ax[1].plot (times, 2 + onset_env / onset_env.max(), alpha=0.8, label= 'Mean (mel)')
onset_env = librosa.onset.onset_strength(y=y, sr=sr, aggregate=np.median, fmax=8000, n_mels=256)
ax[1].plot (times, 1 + onset_env / onset_env.max(), alpha=0.8, label='Median (custom mel)')
C = np.abs (librosa.cqt(y=y,sr=sr) )
onset_env = librosa.onset.onset_strength(sr=sr, S=librosa.amplitude_to_db(C, ref=np.max) )
ax[1].plot(times, onset_env/ onset_env.max(), alpha=0.8, label='Mean (CQT)')
ax[1].legend()
ax[1].set(ylabel=' Normalized strength', yticks=[])

In [ ]:
ra = np.convolve(onset_env, np.ones (100) ,'valid') / 100
sec = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
ra.size

In [ ]:
plt.scatter(np.arange(0,ra.size), ra, s=None, c=None, marker=None, cmap=None, vmin=None, vmax=None, alpha=None, linewidths=None, edgecolors=None)

In [ ]:
soundfile.write (librosa.example('fishin'), y,sr)

The peaks align with the vocals and the valleys show where the audio is only instrumentals.